In [ ]:
import sqlite3
import pandas as pd
import soccerdata as sd
from pathlib import PosixPath
import plotly.express as px

In [ ]:
from database_io.db_handler import DB_handler

In [ ]:
db_handler = DB_handler("/home/morten/Develop/packing-report/gde/GDE.db")

In [ ]:
import sqlite3
import pandas as pd

# Connect to the database
conn = sqlite3.connect("/home/morten/Develop/packing-report/Goal-Difference-Elo-GDE/GDE.db")

# Execute SQL query and load results into a DataFrame
df = pd.read_sql_query("SELECT * FROM games", conn)

In [ ]:
ws = sd.WhoScored(
    #  "GER-Bundesliga", "GER-Bundesliga2", "ENG-Premier League", "ESP-La Liga", "FRA-Ligue 1", "ITA-Serie A"
    leagues=["ENG-Premier League"],
    seasons=[16],
    no_cache=False,
    no_store=False,
    data_dir=PosixPath("/home/morten/Develop/Open-Data/soccerdata"),
    path_to_browser="/usr/bin/chromium",
    headless=True,
)

In [ ]:
events = ws.read_events(match_id=[1080506])
loader = ws.read_events(match_id=[1080506], output_fmt='loader')
loader_players_df = loader.players(1080506)
df_teams = loader.teams(game_id=1080506)

In [ ]:
events

In [ ]:
def is_own_goal(qualifiers):
    return [max([(True if x["type"]["displayName"] == "OwnGoal" else False) for x in events], default=False) for events in qualifiers]

def get_opposition_team(df_goals: pd.DataFrame, df_teams: pd.DataFrame):
    switched_teams = df_goals.copy()
    team_id_one = df_teams["team_id"].unique()[0]
    team_id_two = df_teams["team_id"].unique()[1]
    switched_teams.replace({team_id_one : team_id_two,
                            team_id_two : team_id_one}, inplace = True)
    return switched_teams

def get_score(events_df: pd.DataFrame, df_teams: pd.DataFrame):
    goals = events_df.loc[(events_df["is_goal"] == True)].copy()
    goals["own_goal"] = is_own_goal(goals["qualifiers"])
    goals.loc[~goals["own_goal"], "goal_team_id"] = goals.loc[~goals["own_goal"],"team_id"]
    goals.loc[goals["own_goal"], "goal_team_id"] = get_opposition_team(goals["team_id"], df_teams)[goals["own_goal"]]
    goals.reset_index(inplace=True)
    return goals[["expanded_minute", "goal_team_id"]]

In [ ]:
get_score(events, df_teams).sort_values(by="expanded_minute")

In [ ]:
from gde_utils.date_utils import to_datetime
from datetime import date
from game.game_timeline import GameTimeline

In [ ]:
date = date(2016, 8, 14)
game_timeline = GameTimeline(ws, 1080506, date, "ENG-Premier League", db_handler)
game_timeline.handle()

In [ ]:
df

In [ ]:
px.scatter(df.groupby("game_date").aggregate({"elo": "mean", "opposition_elo": "mean"}).reset_index(), x="game_date", y="elo")

In [ ]:
df[df["starter"] == 1].roundend_expected_game_result.hist()

In [ ]:
df

In [ ]:
df[df["starter"] == 1].roundend_expected_game_result.hist()